go to yelp reviews
zareeens a
yelp.com/dataset
help to see the dataset
we are going to decide the features we are trying to do classification and meassure how the keywords would like or dislike. we need to decode the text to figure out the like or dislike of the review. try sentiment analysis and figure out what each text means if normal, great or low

In [15]:
import numpy as np
import pandas as pd
import json as json
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
import re
from nltk.corpus import stopwords
%matplotlib inline

In [16]:
'''
###### Business json example ######

{'business_id': 'BvYU3jvGd0TJ7IyZdfiN2Q', 
'name': "Manzetti's Tavern", 
'address': '6401 Morrison Blvd', 
'city': 'Charlotte', 
'state': 'NC', 
'postal_code': '28211', 
'latitude': 35.156338, 
'longitude': -80.831878, 
'stars': 3.5, 
'review_count': 16, 
'is_open': 0, 
'attributes': {'Alcohol': "u'full_bar'", 
                'BusinessAcceptsCreditCards': 'True', 
                'RestaurantsAttire': "u'casual'", 
                'RestaurantsTakeOut': 'True', 
                'RestaurantsReservations': 'True', 
                'GoodForKids': 'False', 
                'RestaurantsGoodForGroups': 'True', 
                'BusinessParking': "{'garage': False, 
                                    'street': False, 
                                    'validated': False, 
                                    'lot': True, 
                                    'valet': False}", 
                'OutdoorSeating': 'False', 
                'RestaurantsDelivery': 'False', 
                'RestaurantsPriceRange2': '2'}, 
'categories': 'Sandwiches, Italian, American (Traditional), American (New), Seafood, Steakhouses, Restaurants', 
'hours': {'Monday': '11:0-22:0', 
        'Tuesday': '11:0-22:0', 
        'Wednesday': '11:0-23:0', 
        'Thursday': '11:0-23:0', 
        'Friday': '11:0-23:0', 
        'Saturday': '11:0-23:0', 
        'Sunday': '11:0-22:0'}}
'''

'''
###### Review json example ######

{'review_id': 'Q1sbwvVQXV2734tPgoKj4Q', 
'user_id': 'hG7b0MtEbXx5QzbzE6C_VA', 
'business_id': 'ujmEBvifdJM6h6RLv4wQIg', 
'stars': 1.0, 
'useful': 6, 
'funny': 1, 
'cool': 0, 
'text': 'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', 
'date': '2013-05-07 04:34:36'}

'''

"\n###### Review json example ######\n\n{'review_id': 'Q1sbwvVQXV2734tPgoKj4Q', \n'user_id': 'hG7b0MtEbXx5QzbzE6C_VA', \n'business_id': 'ujmEBvifdJM6h6RLv4wQIg', \n'stars': 1.0, \n'useful': 6, \n'funny': 1, \n'cool': 0, \n'text': 'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', \n'date': '2013-05-07 04:34:36'}\n\n"

In [41]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(\-)|(\/)")

def preprocess_reviews(review):
    review = REPLACE_NO_SPACE.sub("",review.lower())
    review = REPLACE_WITH_SPACE.sub("",review)
    english_stop_words = stopwords.words('english')
    removed_stop_words = ' '.join([word for word in review.split() if word not in english_stop_words])                              
    return removed_stop_words

'''
['02342800\t+Effect\tthrive,prosper,fly_high,flourish\t make steady progress; be at the high point in one\'s career or reach a high point in historical significance or importance; "The new student is thriving"  ']
'''

def create_sentiment_dictionary():
    file1 = './data/effectwordnet/goldStandard.tff'
    file2 = './data/effectwordnet/EffectWordNet.tff'
    temp_dict = {}
    temp_dict_gs = {}
    with open(file2) as fin:
        for line in fin.readlines():
            temp_line = line.splitlines()
            info_list = temp_line[0].split()
            for word in info_list[2].split(','):
                if info_list[1] == '+Effect':
                    temp_dict[word] = 5
                elif info_list[1] == '-Effect':
                    temp_dict[word] = 1
                else:
                    temp_dict[word] = 3
    with open(file1) as fin:
        for line in fin.readlines():
            temp_line = line.splitlines()
            info_list = temp_line[0].split()
            for word in info_list[2].split(','):
                if info_list[1] == '+Effect':
                    temp_dict_gs[word] = 5
                elif info_list[1] == '-Effect':
                    temp_dict_gs[word] = 1
                else:
                    temp_dict_gs[word] = 3
                    
    return temp_dict, temp_dict_gs

In [19]:
json_business_file_test_path = "./data/yelp_dataset/business_test.json"
json_business_file_path = "./data/yelp_dataset/business.json"
json_review_file_path = "./data/yelp_dataset/review.json"

with open(json_business_file_path) as fin:
    business_list = {}
    for number, line in enumerate(fin):
        line_contents = json.loads(line)
        business_list[line_contents['business_id']] = line_contents
        business_list.get(line_contents['business_id'])['reviews'] = []
        business_list.get(line_contents['business_id'])['um_reviews'] = 0
#         if number == 100:
#             break


with open(json_review_file_path) as fin:
    for number, line in enumerate(fin):
        line_contents = json.loads(line)
        business_list.get(line_contents['business_id']).get('reviews').append(line_contents)
#         print('line: {}\n'.format(line_contents))
#         if number == 100:
#             break



In [20]:
sentiment_word_dict, sentiment_word_dict_gs = create_sentiment_dictionary()

In [64]:
test_business_list = []
for business in business_list:
    if business_list.get(business).get('review_count') > 1000:
        #print(business)
        #print(business_list.get(business).get('review_count'))
        test_business_list.append(business)
    else:
        continue

In [65]:
def get_reviews_list(business):
    review_list = []
    score_list = []
    total_num_rating = 0
    actual_rating = business_list.get(business).get('stars')
    for review in business_list.get(business).get('reviews'):
        total_prediction = 0
        num_matches = 0
        temp_score = 0
        temp_text = preprocess_reviews(review.get('text'))
        tokenize_text = temp_text.split()
        for word in tokenize_text:
            if sentiment_word_dict.get(word):
                temp_score += sentiment_word_dict.get(word)
                num_matches += 1
            elif sentiment_word_dict_gs.get(word):
                temp_score += sentiment_word_dict_gs.get(word)
                num_matches += 1

        if not int(num_matches) == 0:
#             print('review_avg_score: {}'.format(temp_score/num_matches))
#             print('num_matches: {}'.format(num_matches))
            review_list.append(temp_text)
            total_num_rating += (temp_score/num_matches)
            if (temp_score/num_matches) < 3:
                score_list.append('negative')
            elif (temp_score/num_matches) > 3:
                score_list.append('positive')
            else:
                score_list.append('neutral')
        
                  
    return_obj = {}
    return_obj['reviews'] = review_list
    return_obj['scores'] = score_list
    return return_obj
        
# print('business name: {}'.format(business_list.get('K7lWdNUhCbcnEvI0NhGewg').get('name')))      
# print('predicted stars: {}'.format(total_num_rating/business_list.get('K7lWdNUhCbcnEvI0NhGewg').get('review_count')))
# print('actual stars: {}\n'.format(actual_rating))

In [66]:

review_data = []
review_score = []

for business in test_business_list:
    res = get_reviews_list(business)
    review_data = review_data + res.get('reviews')
    review_score = review_score + res.get('scores')


print('size of list: {}'.format(len(review_data)))
print('size of label: {}'.format(len(review_score)))
# print(review_data[1])
# print(review_score[1])
# review_score
        

size of list: 632047
size of label: 632047


In [67]:
def transform_to_features(data):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(
        analyzer = 'word',
        lowercase = False,
    )
    features = vectorizer.fit_transform(
        data
    )
    features_nd = features.toarray()
    return features_nd

In [68]:
def train_then_build_model(x_train, x_test, y_train, y_test):

    from sklearn.linear_model import LogisticRegression
    log_model = LogisticRegression()

    log_model = log_model.fit(X=x_train, y=y_train)
    y_pred = log_model.predict(x_test)

    # print first 10th prediction in this format:
    # ::{prediction}::{tweet}
#     for i in range(0,10):
#         index = features_nd.tolist().index(X_test[i].tolist())
#         print("::{}::{}".format(y_pred[i], data[index]))
    
    # print accuracy
    from sklearn.metrics import accuracy_score
    print("Accuracy={}".format(accuracy_score(y_test, y_pred)))

In [69]:
x_train = transform_to_features(x_train_raw)
x_test = transform_to_features(x_test_raw)
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))
# train_then_build_model(x_train, x_test, y_train, y_test )

33628
30372
6869
6869


In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(review_data)
X = tfidf_vectorizer.transform(review_data)
X_test = tfidf_vectorizer.transform(review_data)

X_train, X_val, y_train, y_val = train_test_split(
    X, review_score, train_size = 0.75
)
    
lr = LogisticRegression(C=1)
lr.fit(X_train, y_train)
print ("Accuracy for C=%s: %s" 
       % (c, accuracy_score(y_val, lr.predict(X_val))))

print("Logistic Regression Score: ", cross_val_score(lr, X_train, y_train, cv=3))

# Accuracy Score

y_pred = lr.predict(X_val)
print('Accuracy: ', accuracy_score(y_val, y_pred))

# Precision Score

print('Precision: ', precision_score(y_val, y_pred, average='macro'))

# Recall

print('Recall: ', recall_score(y_val, y_pred, average='macro'))

# F1 Score

print('F1:', f1_score(y_val, y_pred, average='macro'))

/Users/wbaron/Quali/py3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/Users/wbaron/Quali/py3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/wbaron/Quali/py3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy for C=1: 0.9000202516264587


/Users/wbaron/Quali/py3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/wbaron/Quali/py3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression Score:  [0.88658456 0.8878313  0.88702052]
Accuracy:  0.9000202516264587
Precision:  0.9052832950072043
Recall:  0.8840784335104866
F1: 0.8881593159265293


In [90]:
def get_prediction_starts_for_restaurant(review_predictions):
    predicted_stars = -1
    prediction_aggregate_score = 0
    for prediction in review_predictions:
        if prediction == 'positive':
            prediction_aggregate_score += 5
        elif prediction == 'neutral':
            prediction_aggregate_score += 3
        elif prediction == 'negative':
            prediction_aggregate_score += 1
        else:
            print('no rating based on error!!!')
            print(prediction)
            
    predicted_stars = prediction_aggregate_score/len(review_predictions)
    return predicted_stars
            

In [91]:
business_prediction_rank = {}
for business in business_list:
    if business_list.get(business).get('review_count') > 2000:
        actual_rating = business_list.get(business).get('stars')
        business_reviews = []
        for review in business_list.get(business).get('reviews'):
            temp_text = preprocess_reviews(review.get('text'))
            business_reviews.append(temp_text)
            
        review_matrix = tfidf_vectorizer.transform(business_reviews)
        business_predictions = lr.predict(review_matrix)
        predicted_stars = get_prediction_starts_for_restaurant(business_predictions)
        business_predictions_rank[business] = abs(predicted_stars - actual_rating)
        business_list.get(business)['predicted_stars'] = predicted_stars
#         print(business_list.get(business).get('name'))
#         print('prediction score: {}'.format(predicted_stars))
#         print('actual score: {}'.format(actual_rating))
for key, value in sorted(business_prediction_rank.iteritems(), key=lambda (k,v):(v,k)):
    print("**********************************************")
    print("{}: {}".format(business_list.get(key).get('name'), value))
    print("   - Prediction stars: {}".format(business_list.get(key).get('predicted_stars')))
    print("   - Actual stars: {}".format(business_list.get(key).get('stars')))
    print("   - Number of reviews: {}".format(business_list.get(key).get('review_count')))
    print('\n')
        
            
        

SyntaxError: invalid syntax (<ipython-input-91-e00aba7c2f00>, line 18)